In [1]:
from datascience import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import json

In [3]:
data = Table.read_table('data/data_attempts/FY16_External_Funding_by_Department_and_Sponsor_Class.csv')
data

Academic Research Units,Amount
Center for Middle Eastern Studies,"$ 123,354"
Federal,"$ 123,354"
CLAS Center for Latin American Studies,"$ 45,000"
Not for Profit,"$ 45,000"
CSAS Center for South Asian Studies,"$ 53,500"
Federal,$ 0
Non Federal Governmental,$ 0
Not for Profit,"$ 53,500"
CSEAS Center for Southeast Asian Studies,$ 0
Federal,$ 0


In [ ]:
sponsors = ["Federal", "Not for Profit", "Non Federal Governmental", "Industry", 
            "University of California", "State of California"]

In [ ]:
intize_amt = lambda amt: int(amt.split("$")[1].strip().replace(',', ''))
data = data.with_column("Amount", data.apply(intize_amt, "Amount"))
data

In [ ]:
data.column('Amount')[0].split("$")[1].strip()


# New Start

In [6]:
# 10/08 Note: research_fund_comb.csv is a complete csv of the initial pdf file 
combined_t = Table.read_table('data/data_attempts/research_fund_comb.csv').drop([7, 8, 9, 10])
combined_t.show(20)

﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati..,"$75,000","$400,000",nan
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc..,$0,$0,nan
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,Andrew W Mellon Foundation,"$150,000",$0,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,LYRASIS,nan,"$100,018","$70,854"
Campus Support,Vice Chancellor - Administration,Information Services & Technology,NSF National Science Founda..,nan,nan,"$399,446"
Campus Support,Vice Chancellor - Administration,International House,Middlebury College,nan,"$10,000",nan
Campus Support,Vice Chancellor - Administration,Public Safety,Bay Areas Urban Area Securit..,"$327,226","$30,145",nan
Campus Support,Vice Chancellor - Real Estate,Physical Plant Campus Services,California Emergency Manage..,nan,nan,"$573,828"
Colleges,CNR College of Natural Resources,Null,"Abt Associates, Inc.","$73,942",nan,nan


In [7]:
combined_t.select(['2014'])[0] #  parentheses = negative

array(['$75,000 ', '$0 ', '$0 ', ..., '$181,126 ', 'nan', '$950,000 '], 
      dtype='<U13')

In [11]:
t = "Bay Areas Urban Area Securit"
q = "California Emergency Manage"
v = "NSF National Science Founda"
s = "California Department of Indu"
print(len(t), len(q), len(v), len(s))

28 27 27 29


In [54]:
# 10/09 Note: All sponsors downloaded from the Cal Phoebe site 
phoebe = Table.read_table("data/data_attempts/PhoebeSearchResults.csv")
phoebe

Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor,Sponsor Acronym
Federal,Other Federal Category,All Other Federal,000150,CIA Central Intelligence Agency,CIA
Federal,Other Federal Category,All Other Federal,005510,Federal Mediation & Conciliation Service,FMCS
Federal,Other Federal Category,All Other Federal,005440,National Commission on Air Quality,nan
Federal,Other Federal Category,All Other Federal,005350,VA Department of Veterans Affairs,VA
Federal,Other Federal Category,All Other Federal,005325,Smithsonian - National Museum Natural History,nan
Federal,Other Federal Category,All Other Federal,005305,Smithsonian - Office of Fellowships & Grants,nan
Federal,Other Federal Category,All Other Federal,005302,Smithsonian - Office of Environmental Sciences,nan
Federal,Other Federal Category,All Other Federal,005301,Smithsonian Institution,nan
Federal,Other Federal Category,All Other Federal,005290,National Archives and Records Administration,nan
Federal,Other Federal Category,All Other Federal,005285,SBA Small Business Administration,SBA


In [55]:
phoebe_trunc = phoebe.with_column("SponsorName_trunc", 
                                 phoebe.apply(lambda x: x[0:28], "Sponsor"))
phoebe_trunc

Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor,Sponsor Acronym,SponsorName_trunc
Federal,Other Federal Category,All Other Federal,000150,CIA Central Intelligence Agency,CIA,CIA Central Intelligence Age
Federal,Other Federal Category,All Other Federal,005510,Federal Mediation & Conciliation Service,FMCS,Federal Mediation & Concilia
Federal,Other Federal Category,All Other Federal,005440,National Commission on Air Quality,nan,National Commission on Air Q
Federal,Other Federal Category,All Other Federal,005350,VA Department of Veterans Affairs,VA,VA Department of Veterans Af
Federal,Other Federal Category,All Other Federal,005325,Smithsonian - National Museum Natural History,nan,Smithsonian - National Museu
Federal,Other Federal Category,All Other Federal,005305,Smithsonian - Office of Fellowships & Grants,nan,Smithsonian - Office of Fell
Federal,Other Federal Category,All Other Federal,005302,Smithsonian - Office of Environmental Sciences,nan,Smithsonian - Office of Envi
Federal,Other Federal Category,All Other Federal,005301,Smithsonian Institution,nan,Smithsonian Institution
Federal,Other Federal Category,All Other Federal,005290,National Archives and Records Administration,nan,National Archives and Record
Federal,Other Federal Category,All Other Federal,005285,SBA Small Business Administration,SBA,SBA Small Business Administr


In [12]:
combined_clean_name = combined_t.with_column('Direct Sponsor', 
                                        combined_t.apply(lambda x: x.split(".")[0], 'Direct Sponsor'))
combined_clean_name

﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati,"$75,000","$400,000",nan
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc,$0,$0,nan
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,Andrew W Mellon Foundation,"$150,000",$0,nan
Campus Support,Vice Chancellor - Administration,Information Services & Technology,LYRASIS,nan,"$100,018","$70,854"
Campus Support,Vice Chancellor - Administration,Information Services & Technology,NSF National Science Founda,nan,nan,"$399,446"
Campus Support,Vice Chancellor - Administration,International House,Middlebury College,nan,"$10,000",nan
Campus Support,Vice Chancellor - Administration,Public Safety,Bay Areas Urban Area Securit,"$327,226","$30,145",nan
Campus Support,Vice Chancellor - Real Estate,Physical Plant Campus Services,California Emergency Manage,nan,nan,"$573,828"
Colleges,CNR College of Natural Resources,Null,"Abt Associates, Inc","$73,942",nan,nan


In [ ]:
# Doesn't work well 
import difflib
# df2.index = df2.index.map(lambda x: difflib.get_close_matches(x, df1.index)[0])
combined_clean_name.apply(lambda x: difflib.get_close_matches(x, phoebe_trunc.column("Sponsor")), "Direct Sponsor")

In [56]:
# combined_clean_name.join("Direct Sponsor", phoebe_trunc, "SponsorName_trunc")
# 10/09 Note: Finding the corresponding "complete" names of sponsors
extended_names = {}
remnants = {}
for i in range(combined_clean_name.num_rows):
    source_name = combined_clean_name.column('Direct Sponsor')[i]
    target = phoebe_trunc.column("Sponsor")
    for j in range(len(target)):
        target_name = target[j]
        if (i) not in extended_names.keys() and source_name in target_name:
            extended_names[(i)] = target_name

In [57]:
extended_names

{0: 'LBNL Lawrence Berkeley National Laboratory',
 1: 'SAMHSA Center for Substance Abuse Prevention',
 2: 'UC Office of the President',
 3: 'Andrew W Mellon Foundation',
 4: 'LYRASIS',
 5: 'NSF National Science Foundation',
 6: 'Middlebury College',
 7: 'Bay Areas Urban Area Security Initiative',
 8: 'California Emergency Management Agency',
 9: 'Abt Associates, Inc.',
 10: 'California Department of Forestry and Fire Protection',
 11: 'NIH Office of the Director',
 12: 'Oregon, State of',
 13: 'Research Triangle Institute',
 14: 'UC Davis',
 15: 'UC San Francisco',
 16: 'USDA Forest Service',
 17: 'California Department of Industrial Relations',
 18: 'California Department of Water Resources',
 19: 'California Dried Plum Board',
 20: 'California Energy Commission',
 21: 'California Polytechnic State University Foundation',
 22: 'California Public Utilities Commission',
 23: 'Cotton Inc.',
 24: 'California Department of Resources Recycling and Recovery (DO NOT USE - USE 00F428 Instead)'

In [28]:
combined_t.column('Direct Sponsor')[991]

'German Academic Exchange ..'

In [58]:
orig = (np.array(list(extended_names.keys())))
missing = []
for elem in range(combined_clean_name.num_rows):
    if elem not in orig:
        missing += [elem]
print("Row IDs of sponsors in combined_clean_name not found", missing)

Row IDs of sponsors in combined_clean_name not found [1434, 2332]


In [37]:
combined_clean_name = combined_clean_name.with_column('index', range(combined_clean_name.num_rows))
combined_clean_name.show(3)

﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016,index
Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati,"$75,000","$400,000",nan,0
Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc,$0,$0,nan,1
Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan,2


In [59]:
combined_clean_name.row([1434, 2332])

Row(﻿Control Unit=array(['L&S College of Letters & Sciences', 'Vice Chancellor - Research'], 
      dtype='<U40'), Division=array(['L&S Biological Sciences', 'Academic Research Units'], 
      dtype='<U47'), Department=array(['MCB Molecular & Cell Biology',
       'IIS Institute of International Studies'], 
      dtype='<U60'), Direct Sponsor=array(['Helmholtz Zentrum MÃ¼nchen', 'JOhn D and Catherine T MacA'], 
      dtype='<U33'), 2014=array(['nan', 'nan'], 
      dtype='<U13'), 2015=array(['nan', '$140,000 '], 
      dtype='<U12'), 2016=array(['$0 ', 'nan'], 
      dtype='<U32'), index=array([1434, 2332]))

In [60]:
#adding the leftover 

# 10/09 Note: These two names are found in the corresponding rows of combined_clean_name but were either mispelled 
# or warped 
extended_names[1434] = "Helmholtz Zentrum München"
extended_names[2332] = "John D and Catherine T MacArthur Foundation"

In [61]:
len(extended_names) 

2871

In [63]:
# Disregard
combined_clean_name.column('Direct Sponsor')[1]

'SAMHSA Center for Substanc'

In [ ]:
# Disregard

naive_combined = combined_clean_name.join('Direct Sponsor', phoebe_trunc, "Sponsor")
naive_combined.show(5)

In [ ]:
# Disregard

all_combined_sponsors = list(combined_clean_name.column('Direct Sponsor'))
naive_combined_sponsors = list(naive_combined.column('Direct Sponsor'))
remnant_sponsors = []

for i in range(len(all_combined_sponsors)):
    if all_combined_sponsors[i] not in naive_combined_sponsors:
        remnant_sponsors += [all_combined_sponsors[i] ]

In [ ]:
# Disregard

# another attempt 
combined_clean_name = combined_clean_name.with_column('len', 
                                combined_clean_name.apply(lambda x: len(x), 'Direct Sponsor'))
combined_clean_name.show(3)

## Starting with extended_names (as a means to join phoebe and data)

In [64]:
extended_names_tbl = Table().with_columns(['index', extended_names.keys(),
                                           'Sponsor', extended_names.values()])
extended_names_tbl.show(2)

index,Sponsor
0,LBNL Lawrence Berkeley National Laboratory
1,SAMHSA Center for Substance Abuse Prevention


In [65]:
combined_clean_name = combined_clean_name.join('index', extended_names_tbl)
combined_clean_name.show(3)

index,﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016,Sponsor
0,Campus Support,Vice Chancellor - Administration,Finance,LBNL Lawrence Berkeley Nati,"$75,000","$400,000",nan,LBNL Lawrence Berkeley National Laboratory
1,Campus Support,Vice Chancellor - Administration,Finance,SAMHSA Center for Substanc,$0,$0,nan,SAMHSA Center for Substance Abuse Prevention
2,Campus Support,Vice Chancellor - Administration,Finance,UC Office of the President,$0,nan,nan,UC Office of the President


In [66]:
combined_phoebe = combined_clean_name.join('Sponsor', phoebe)
combined_phoebe.show(5)

Sponsor,index,﻿Control Unit,Division,Department,Direct Sponsor,2014,2015,2016,Sponsor Class,Sponsor Category,Sponsor Type,Sponsor Code,Sponsor Acronym
"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",1471,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,NIH National Center for Rese,nan,$0,nan,Federal,DHHS,DHHS-NIH,003417,NCRR
3M Corporation,559,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,3M Corporation,nan,"$250,000",nan,Industry,USA Industry Category,USA Industry,001003,3M
"4D Molecular Therapeutics, LLC",2534,Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),"4D Molecular Therapeutics, L",nan,nan,"$22,408",Industry,USA Industry Category,USA Industry,00VC9H,nan
"AFINGEN, Inc.",2537,Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),"AFINGEN, Inc",nan,nan,"$5,000",Industry,USA Industry Category,USA Industry,00VBJK,nan
"APATH, L.C.C.",1391,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,"APATH, L",$0,nan,nan,Industry,USA Industry Category,USA Industry,004928,nan


In [70]:
combined_phoebe_conc = combined_phoebe.select(['Sponsor', '\ufeffControl Unit', 'Division', 'Department', 'Sponsor Class', '2014', '2015', '2016'])
combined_phoebe_conc.show(3)
# combined_phoebe.labels

Sponsor,﻿Control Unit,Division,Department,Sponsor Class,2014,2015,2016
"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,nan,$0,nan
3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,nan,"$250,000",nan
"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,nan,nan,"$22,408"


In [ ]:
# combined_phoebe_conc.to_csv('data/combined_phoebe_conc.csv')

In [71]:
combined_phoebe_conc_pd = pd.read_csv('data/data_attempts/combined_phoebe_conc.csv')
combined_phoebe_conc_pd

,Sponsor,Control Unit,Division,Department,Sponsor Class,2014,2015,2016
0,"""DEACTIVATED - use 00V8VH"" NIH National Center...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,NaN,$0,NaN
1,3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,NaN,"$250,000",NaN
2,"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Bios...,Industry,NaN,NaN,"$22,408"
3,"AFINGEN, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Bios...,Industry,NaN,NaN,"$5,000"
4,"APATH, L.C.C.",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Industry,$0,NaN,NaN
5,AYUSA International,Vice Chancellor - Research,Academic Research Units,IURD Institute of Urban & Regional Development,Not for Profit,"$54,481","$60,000",NaN
6,Aalborg University Copenhagen,Vice Chancellor - Research,Vice Chancellor for Research ORUs,IBER Institute of Business & Economic Research,Not for Profit,NaN,"$53,015",NaN
7,Abraham J. & Phyllis Katz Foundation,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Not for Profit,$0,NaN,NaN
8,"Abt Associates, Inc.",Colleges,CNR College of Natural Resources,Null,Not for Profit,"$73,942",NaN,NaN
9,Academy of Applied Science,Colleges,CNR College of Natural Resources,"Environmental Science, Policy & Management",Not for Profit,NaN,"$4,000","$4,000"


In [ ]:
# 10/09 Note: Beginning to  clean dollar amounts
combined_phoebe_conc_pd = combined_phoebe_conc_pd.fillna("Null")

In [ ]:
test = '$5,418'
eval(test.replace('$', '').replace(',', ''))

In [ ]:
# dealing with parentheses and dollar signs 
def money_parse(strval):
    if strval == "Null":
        return 0
    if "(" in strval:
        # Parentheses signify negatives 
        parsed = eval(strval.replace('($', '').replace(')', '').replace(',', '')) * -1
    else:
        parsed = eval(strval.replace('$', '').replace(',', ''))
    return parsed 

combined_phoebe_conc_pd['2014'] = combined_phoebe_conc_pd['2014'].apply(money_parse)
combined_phoebe_conc_pd['2015'] = combined_phoebe_conc_pd['2015'].apply(money_parse)
combined_phoebe_conc_pd['2016'] = combined_phoebe_conc_pd['2016'].apply(money_parse)

In [ ]:
combined_phoebe_conc_pd.head(3)

In [ ]:
# 10/09 Note: contains cleaned data and considers negatives 
combined_phoebe_conc_pd.to_csv('data/data_attempts/research_funding_2014_2015_2016.csv')

### Tinkering with visualization methods
#### 10/09 Note: Jump to other notebook, "private-research" 

In [ ]:
# agg over time by department 
combined_phoebe_conc_pd_ds = Table.read_table('data/data_attempts/research_funding_2014_2015_2016.csv')

In [ ]:
sum(combined_phoebe_conc_pd_ds.column('2014')), sum(combined_phoebe_conc_pd_ds.column('2015')), sum(combined_phoebe_conc_pd_ds.column('2016'))

In [ ]:
combined_phoebe_conc_pd_ds.group('Department', sum)

In [ ]:
combined_phoebe_conc_pd_ds.where('Department', 'Null')

In [ ]:
# breakdown of sponsor class by division 
combined_phoebe_conc_pd.groupby('Division').count()

# VIZ: network with division as center and names of donors connected to it, color coded by class 

In [ ]:
# sum of funds over 3 years by division 
combined_phoebe_conc_pd.groupby('Division').sum()

In [ ]:
ds_above = Table.read_table('rfund_byDivision_sumByYear.csv')
ds_above

In [ ]:
div = ds_above.column(0)
fy14 = ds_above.column(1)
fy15 = ds_above.column(2)
fy16 = ds_above.column(3)
fy141516 = zip(div, zip(fy14, fy15, fy16))

In [ ]:
fy141516_t = Table().with_columns(list(fy141516))

In [ ]:
fy141516_t.to_csv("rfund_byDivision_sumByYear_opp_format.csv")

In [ ]:
pd_above = pd.read_csv('rfund_byDivision_sumByYear_opp_format.csv')
pd_above.plot.line()

In [ ]:
fy141516_t

In [ ]:
# combined_phoebe_conc_pd_ds = combined_phoebe_conc_pd_ds.relabel("Sponsor Class", "Sponsor_class")
network_comb = combined_phoebe_conc_pd_ds.select("Sponsor", "Department", "Sponsor Class", "total_given").relabel("Sponsor Class", "Sponsor_class")
network_comb

In [ ]:
# Network File / Edge List
network_comb.to_csv("network_rfund.csv")

In [ ]:
network_comb_edited = Table.read_table("network_rfund.csv")

In [ ]:
combined_phoebe_conc_pd_ds = combined_phoebe_conc_pd_ds.with_column('total_given', 
                                        combined_phoebe_conc_pd_ds.apply(lambda x, y, z: x + y + z, ['2014', '2015', '2016']))

In [ ]:
combined_phoebe_conc_pd_ds.group

In [ ]:
# network_comb into json 
groups = {"Federal": 1, "Industry": 2, "Not for Profit": 3, "University of California": 4, "Non Federal Governmental": 5}
def nodify(table, colname, rdict, groupmapping):
    node_lst = []
    nodified = []
    col = table.column(colname)
    sclass = table.column("Sponsor_class")
    for i in range(len(col)):
        thisdict = {}
        if (col[i] not in nodified):
            thisdict["id"] = col[i] 
            nodified += [col[i]]
            if colname == "Department":
                thisdict["group"] = 0 #just for department, sponsor group will vary depending on sponsor_class
            else:
                thisdict["group"] = groupmapping[sclass[i]]
            node_lst += [thisdict]
    rdict["nodes"] = node_lst
    return "Done"
        

In [ ]:
network_comb_edited.group('Department').num_rows + network_comb_edited.group('Sponsor').num_rows

In [ ]:
test = {}
nodify(network_comb_edited, "Department", test, groups)

In [ ]:
sponsor_nodes = {}
nodify(network_comb_edited, "Sponsor", sponsor_nodes, groups)

In [ ]:
all_nodes = {}
all_nodes["nodes"] = test["nodes"] + sponsor_nodes["nodes"]

In [ ]:
def edgify(table, source, target, value, rdict):
    source_col = table.column(source)
    target_col = table.column(target)
    value_col = table.column(value)
    links = []
    for i in range(table.num_rows):
        thisdict = {}
        s = source_col[i]
        t = target_col[i]
        v = value_col[i]
        thisdict["source"] = s
        thisdict["target"] = t
        thisdict["value"] = v
        links += [thisdict]
    rdict["links"] = links 
    return "Done"

In [ ]:
edgify(network_comb_edited, "Sponsor", "Department", "total_given", all_nodes)

In [ ]:
len(all_nodes['nodes']), network_comb_edited.num_rows

In [ ]:
# writing all_nodes to file
file = open("rfund_nodes_links.txt", "w")
file.write(str(all_nodes))
file.close()

In [ ]:
import ast

file = open("rfund_nodes_links.txt", "r+")
file.write((json.dumps(ast.literal_eval(str(all_nodes)))))
file.close()

In [ ]:
json.loads((json.dumps(ast.literal_eval(str(all_nodes)))))

In [ ]:
print(json.dumps(ast.literal_eval(str(all_nodes)), sort_keys=True, indent=4))

In [ ]:
import numpy as np
network_comb_edited.group('Department')
# Astronomy
# Anthropology
testing = network_comb_edited.where(np.logical_or(network_comb_edited.column('Department') == 'Astronomy', \
                                                  network_comb_edited.column('Department') == 'Anthropology'))

In [ ]:
testing

In [ ]:
testing_nodes = {}
nodify(testing, "Sponsor", testing_nodes, groups)
testing_nodes_dept = {}
nodify(testing, "Department", testing_nodes_dept, groups)
testing_nodes_all = {}
testing_nodes_all["nodes"] = testing_nodes_dept["nodes"] + testing_nodes["nodes"]

In [ ]:
edgify(testing, "Sponsor", "Department", "total_given", testing_nodes_all)

In [ ]:
print(json.dumps(ast.literal_eval(str(testing_nodes_all)), sort_keys=True, indent=4))

In [ ]:
network_comb_edited.group('Sponsor_class')

In [ ]:
# [{"name":"flare.analytics.cluster.AgglomerativeCluster","size":3938,"imports":["flare.animate.T
def hierarchical_edge_bundling(table):
    dept = table.column('Department')
    spon = table.column('Sponsor')
    size = table.column('total_given') # log it and multiply by 10 
    rv = []
    nan = np.log(-1)
    inf = np.log(0)
    for i in range(len(dept)):
        curr = {}
        if (dept[i] != "Null"): 
            curr["name"] = dept[i]
            val = np.log(sum(size[i])) * 10
            if ((val) != nan and (val) != inf):
                curr["size"] = val
            else:
                curr["size"] = 0
            curr["imports"] = list(spon[i])
            rv += [curr]
        else:
            print(dept[i])
    return rv

def heb_sponsor(table):
    spon = table.column('Sponsor')
    rv = []
    for i in range(len(spon)):
        curr = {}
        curr["name"] = spon[i]
        curr["size"] = 10
        curr["imports"] = []
        rv += [curr]
    return rv 

In [ ]:
network_comb_heb = network_comb_edited.group('Department', lambda x: x)
network_comb_heb

In [ ]:
rv = hierarchical_edge_bundling(network_comb_heb)

In [ ]:
import json
print(json.dumps(rv, sort_keys=True, indent=4))
# not working bc each import needs its own node

In [ ]:
rv2 = heb_sponsor(network_comb_edited.group("Sponsor"))

In [ ]:
print(json.dumps(rv2, sort_keys=True, indent=4))

In [ ]:
combined_phoebe_conc_pd_ds.group("Department").column(0)

### What if I split it into multiple hierarchical edge bundling visualizations? 
One for schools and another for other? 

In [ ]:
select_dept = ['Anthropology',
       'Architecture', 'Art History','ARF Archaeological Research Facility',
       'Astronomy', 
       'BNNI Berkeley Nanoscience & Nanoengineering Institute',
       'Berkeley Language Center', 'Berkeley Seismological Laboratory',
       'Bioengineering',
       'Blum Center for Developing Economies', 'Boalt Law School',
        'Buddhist Studies', 'CAL Cal Performances',
       'CEDR Center for Environmental Design Research',
       'CIEE California Institute for Energy & Environment',
       'CITRIS Ctr for Info Tech Research in the Interest of Society',
       'CLAS Center for Latin American Studies',
       'CPAM Center for Pure & Applied Mathematics',
       'CSAS Center for South Asian Studies',
       'CSEAS Center for Southeast Asian Studies',
       'Cancer Research Laboratory', 'Center for African Studies',
       'Center for Child and Youth Policy',
       'Center for Educational Partnerships',
       'Center for Emerging & Neglected Diseases',
       'Center for Integ8rative Planetary Science',
       'Center for Jewish Studies', 'Center for Middle Eastern Studies',
       'Center for Studies in Higher Education',
       'Center of Evaluation for Global Action',
       'Chemical and Biomolecular Engineering', 'Chemistry Department',
       'Civil & Environmental Engineering', 'Classics',
       'Comparative Literature', 'Critical Theory', 'D-Lab', 'Data Science Institute',
        'Demography',
       'Dept of Social Welfare', 'EBI Energy Biosciences Institute',
       'EECS Electrical Engineering & Computer Sciences',
       'EERC Earthquake Engineering Research Center',
       'EHS Environment, Health & Safety',
       'Earth & Planetary Science', 'East Asian Languages', 'Economics',
       'Energy & Resources Group', 'Energy and Climate Institute',
       'English', 'Environmental Science, Policy & Management',
       'Essig Museum of Entomology', 'Ethnic Studies', 'Finance',
       'Functional Genomics Laboratory',
       'Geography', 'German',
       'Gump Research Field Station', 'Haas Centers and Institutes',
       'Haas Diversity Research Center (HIFIS Haas Institute)',
       'Haas School of Business (Department)',
       'Helen Wills Neuroscience Institute', 'History',
       'History of Science & Technology', 'IB Integrative Biology',
       'IBER Institute of Business & Economic Research',
       'IBI Institute for Business Innovation',
       'ICBS Institute for Cognitive and Brain Sciences',
       'IEAS Institute of East Asian Studies',
       'IEOR Industrial Engineering & Operations Research',
       'IESE Institute for Environmental Science & Engineering',
       'IGS Institute of Governmental Studies',
       'IHD Institute of Human Development',
       'IIS Institute of International Studies',
       'IPIRA Intellectual Property & Industry Research',
       'IRLE Institute for Research on Labor and Employment',
       'ISSI Institute for the Study of Societal Issues',
       'ITS Institute of Transportation Studies',
       'IURD Institute of Urban & Regional Development',
       'Information Services & Technology',
       'Innovative Genomics Initiative',
       'Institute for the Study of Societal Issue',
       'Institute for the Study of Societal Issues',
       'Jewish Graduate Studies', 'Journalism Department',
       'LHS Lawrence Hall of Science', 'Linguistics',
       'MCB Molecular & Cell Biology',
       'MSE Materials Science & Engineering', 'Mathematics',
       'Mechanical Engineering', 'Music', 'NE Near Eastern Studies',
       'Nuclear Engineering', 'Nutritional Science & Toxicology', 'Optometry Department',
        'Physics',
       'Plant & Microbial Biology Department', 'Political Science',
       'Psychology', 'Public Policy', 
       'RAL Radio Astronomy Laboratory',
       'Research Field Stations and Nature Reserves', 'Rhetoric',
       'S&P Spanish & Portuguese', 'SPH Divisional Rsrch and Cntrs',
       'SSEAS South & Southeast Asian Studies', 'SSL Space Sciences Lab',
       'Scandinavian Languages', 'School of Education',
       'School of Journalism, Faculty Research',
       'Simons Institute for the Theory of Computing', 'Social Welfare',
       'Sociology', 'Statistics', 'Stem Cell Center',
       'TAC Theoretical Astrophysics Center',
       'The California Institute for Quantitative Biosciences (QB3)',
       'The Social Science Matrix',
       'Townsend Center for the Humanities', 'UC Energy Institute',
       'UGIS IAS Department',
       "Women's Studies", 'iSchool School of Information']

In [ ]:
select_dept_t = Table().with_column('major_dept', np.array(select_dept))

In [ ]:
major_dept = combined_phoebe_conc_pd_ds.join('Department', select_dept_t, "major_dept")
major_dept.show(3)

In [ ]:
major_dept.group('Sponsor').num_rows + major_dept.group('Department').num_rows

In [ ]:
major_dept =  major_dept.with_column("total_given", 
                                    major_dept.apply(lambda x, y, z: x+y+z, ['2014', '2015', '2016']))

In [ ]:
# research / non-administrative 
rv_selec = hierarchical_edge_bundling(major_dept.group("Department", lambda x: x))

In [ ]:
print(json.dumps(rv_selec, sort_keys=True, indent=4))

In [ ]:
rv_selec2 = heb_sponsor(major_dept.group("Sponsor"))

In [ ]:
print(json.dumps(rv_selec2, sort_keys=True, indent=4))

### The research / nonadministrative didn't work out too hot 

In [ ]:
minor_names = Table.read_table("minor_names.tsv")

In [ ]:
minor_dept = combined_phoebe_conc_pd_ds.join('Department', minor_names, "names")
minor_dept =  minor_dept.with_column("total_given", 
                                    minor_dept.apply(lambda x, y, z: x+y+z, ['2014', '2015', '2016']))

In [ ]:
minor_dept.group('Sponsor').num_rows + minor_dept.group('Department').num_rows

In [ ]:
# research / non-administrative 
rv_admin = hierarchical_edge_bundling(minor_dept.group("Department", lambda x: x))
rv_admin2 = heb_sponsor(minor_dept.group("Sponsor"))
rv_admin_all = rv_admin2 + rv_admin
print(json.dumps(rv_admin_all, sort_keys=True, indent=4))

### Also not pretty and non-informative
#### Next step: top 10 donors in each sponsor class, top 10 department recipients (raw amount)

In [ ]:
combined_phoebe_conc_pd = pd.read_csv('data/data_attempts/research_funding_2014_2015_2016.csv')
combined_phoebe_conc_pd['Total'] = combined_phoebe_conc_pd['2014'] + combined_phoebe_conc_pd['2015'] + combined_phoebe_conc_pd['2016']

In [ ]:
# combined_phoebe_conc_pd['Total'] = combined_phoebe_conc_pd['2014'] + combined_phoebe_conc_pd['2015'] + combined_phoebe_conc_pd['2016']

combined_phoebe_conc_pd.groupby(['Sponsor Class', 'Sponsor'])[['Total']].sum().sort_values('Total', ascending=False).iloc[:10]

In [ ]:
classes = combined_phoebe_conc_pd['Sponsor Class'].unique()
classes

In [ ]:
def separate_classes(df, classes):
    dfs = []
    for classtype in classes:
        sep = df[df['Sponsor Class'] == classtype]
        dfs += [sep]
    return dfs

In [ ]:
dfs = separate_classes(combined_phoebe_conc_pd, classes)

In [ ]:
# Federal 
for i in range(5):
    print("\n", classes[i].upper())
    print(dfs[i].groupby(['Sponsor'])[['Total']].sum().sort_values('Total', ascending=False).iloc[:10])

### Top 10 and Bottom 10 recipients of aid (aggregate) 

In [ ]:
combined_phoebe_conc_pd.head(3)

In [ ]:
top_10_rec = combined_phoebe_conc_pd.groupby(['Control Unit', 'Division', 'Department'])[['Total']].sum()\
                                    .sort_values(by='Total', ascending=False)
top_10_rec.head(12)

# bubble viz or tree map (include the bottom 10)

In [ ]:
bottom_10_rec = combined_phoebe_conc_pd.groupby(['Control Unit', 'Division', 'Department'])[['Total']].sum()\
                                    .sort_values(by='Total', ascending=True)
bottom_10_rec[15:25]

### Will have to think about if I want to include the above top 10 and bottom 10 and how those should be visualized. 

## Onto next: time series (viz 3) 

In [ ]:
by_class_time = combined_phoebe_conc_pd.groupby(['Sponsor Class'])[['2014', '2015', '2016']].sum()
by_class_time

In [ ]:
ax = by_class_time.transpose().plot.line(legend=False)
ax.legend(loc="best", prop={'size': 10})
ax.grid(True)
ax.set_ylabel("Funds ($)", fontsize=8)
ax.set_xlabel("Fiscal Year", fontsize=8)


In [ ]:
# by_class_time.transpose().to_csv('by_class_time.csv')

### Onto Viz 4: network graph by college / school

In [ ]:

def college_school():
    bool_arr = []
    for cu in combined_phoebe_conc_pd['Control Unit']:
        if cu == "L&S College of Letters & Sciences" or cu == "Colleges" or cu == "Schools":
            bool_arr += [True]
        else:
            bool_arr += [False]
    return bool_arr

In [ ]:
bool_arr = college_school()

In [ ]:
bool_series = pd.Series(bool_arr)
college_school_pd = combined_phoebe_conc_pd[bool_series.values]

In [ ]:
ls = college_school_pd[college_school_pd['Control Unit'] == "L&S College of Letters & Sciences"]
coe = college_school_pd[college_school_pd['Division'] == "College of Engineering"]
cnr = college_school_pd[college_school_pd['Division'] == "CNR College of Natural Resources"]
coc = college_school_pd[college_school_pd['Division'].isin(["College of Chemistry", "Chemistry Department"])]
ced = college_school_pd[college_school_pd['Division'] == "College of Environmental Design"]
law = college_school_pd[college_school_pd['Division'] == "School of Law"]
opt = college_school_pd[college_school_pd['Division'] == "School of Optometry"]
sph = college_school_pd[college_school_pd['Division'] == "School of Public Health"]
pp = college_school_pd[college_school_pd['Division'] == "Goldman School of Public Policy"]
haas = college_school_pd[college_school_pd['Division'] == "Haas School of Business (Division)"]
info = college_school_pd[college_school_pd['Division'] == "School of Information"]
edu = college_school_pd[college_school_pd['Division'] == "Graduate School of Education"]
sw = college_school_pd[college_school_pd['Division'] == "School of Social Welfare"]
jour = college_school_pd[college_school_pd['Division'] == "School of Journalism"]

In [ ]:
college_schools_dfs = [ls, coe, cnr, coc, ced, law, opt, sph, pp, haas, info, edu, sw, jour]

In [ ]:
for elem in college_schools_dfs:
    elem

In [ ]:
jour[jour['Total'] > 0].groupby(['Sponsor Class', 'Sponsor'])[['Total']].sum().reset_index().to_csv('rfund_jour.csv')

In [ ]:
jour[jour['Total'] > 0].groupby(['Sponsor Class', 'Sponsor'])[['Total']].sum().reset_index().shape

### Sept Restart

In [ ]:
test = "United Technologies Corp (Incl. UTCR United Tech Res "
# " ".join(test[0:([i for i, x in enumerate(truth) if x])[0]])
truncate_names(test)
# type(['United', 'Technologies', 'Corp', '(Incl.', 'UTCR', 'United', 'Tech', 'Res', ''])
# t1 = "Molecular & Cell Biology (yolo my girl)"
# [True if "(" in x else False for x in t1]
# has_p = [True if x.find("(") != -1 else False for x in test]
# if np.any(has_p):
#     print([i for i, x in enumerate(has_p) if x])

In [2]:
# Method gets rid of extra information contained in parentheses, as well as abbreviations
def truncate_names(name):
    truncated = name.split(" ")
    modified = 0
    
    # abbrev
    if np.all([True if x.isupper() else False for x in truncated[0]]):
        if truncated[1][0] == truncated[0][0]:
            truncated = truncated[1:]
            modified = 1
        else:
            modified = 0
    # parentheses
    has_p = [True if x.find("(") != -1 else False for x in truncated]
    if np.any(has_p):
        index = ([i for i, x in enumerate(has_p) if x][0])
        truncated = " ".join(truncated[0:index])
        modified = 1
    
    if not modified:
        return name
    return " ".join(truncated) if type(truncated) == list else truncated

In [3]:
combined_phoebe_conc_pd_ds = Table().read_table('data/data_attempts/research_funding_FINAL.csv')
combined_phoebe_conc_pd_ds.show(10)

Unnamed: 0,Sponsor,Control Unit,Division,Department,Sponsor Class,2014,2015,2016
0,"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,0,0,0
1,3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,0,250000,0
2,"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,22408
3,"AFINGEN, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,5000
4,"APATH, L.C.C.",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Industry,0,0,0
5,AYUSA International,Vice Chancellor - Research,Academic Research Units,IURD Institute of Urban & Regional Development,Not for Profit,54481,60000,0
6,Aalborg University Copenhagen,Vice Chancellor - Research,Vice Chancellor for Research ORUs,IBER Institute of Business & Economic Research,Not for Profit,0,53015,0
7,Abraham J. & Phyllis Katz Foundation,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Not for Profit,0,0,0
8,"Abt Associates, Inc.",Colleges,CNR College of Natural Resources,Null,Not for Profit,73942,0,0
9,Academy of Applied Science,Colleges,CNR College of Natural Resources,"Environmental Science, Policy & Management",Not for Profit,0,4000,4000


In [4]:
industry = combined_phoebe_conc_pd_ds.where('Sponsor Class', "Industry")
industry = industry.with_column("Total", industry.apply(lambda x, y, z: x+y+z, ['2014', '2015', '2016']))\
                    .select([1, 2, 3, 4, 'Total']).sort('Total', descending=True)

In [5]:
industry_top50 = industry.take(np.arange(50))

In [6]:
industry_other = industry.take(np.arange(50, industry.num_rows))
industry_other.num_rows

399

In [7]:
industry_top50_null = industry_top50.where("Department", "Null").select(["Sponsor", 'Division', "Total"]).relabel("Division", "Department")
industry_top50 = industry_top50.where(industry_top50.column("Department") != "Null").select(['Sponsor','Department', 'Total']).append(industry_top50_null)
industry_null = industry_other.where("Department", "Null").select(["Sponsor", 'Division', "Total"]).relabel("Division", "Department")
industry_other = industry_other.where(industry_other.column("Department") != "Null").select(['Sponsor','Department', 'Total']).append(industry_null)

In [8]:
# For other mapping:
industry_other = industry_other.with_columns(['Sponsor', industry_other.apply(truncate_names, ['Sponsor']),
                                              'Department', industry_other.apply(truncate_names, ['Department'])])\
                               .where('Total', are.above(0))
industry_other.show(3)

Sponsor,Department,Total
Chevron Corporation,Chemistry Department,625000
BASF Corporation,Nutritional Science & Toxicology,619578
Chevron Corporation,Electrical Engineering & Computer Sciences,600000


In [25]:
other_grouped = industry_other.select('Department','Total').group('Department', sum).relabel('Total sum', 'Total')
other_grouped = other_grouped.with_column('Sponsor', ['Other'] * other_grouped.num_rows)
other_grouped

Department,Total,Sponsor
Berkeley Seismological Laboratory,213855,Other
Bioengineering,540849,Other
Boalt Law School,70000,Other
Botanical Garden,52200,Other
CNR Office of the Dean,78349,Other
California Institute for Energy & Environment,565850,Other
Center for Environmental Design Research,2032610,Other
Center for Pure & Applied Mathematics,76635,Other
Center for Studies in Higher Education,50000,Other
Center of Evaluation for Global Action,221552,Other


In [10]:
industry_all = industry_top50.append(other_grouped)

industry_all = industry_all.with_columns(['Sponsor', industry_all.apply(truncate_names, ['Sponsor']),
                                          'Department', industry_all.apply(truncate_names, ['Department'])])
industry_all.num_rows

98

In [27]:
industry_all.sort('Total', descending=True)

Sponsor,Department,Total
BASF Corporation,Chemistry Department,9069312
Other,Electrical Engineering & Computer Sciences,7582226
Aduro Biotech,Cancer Research Laboratory,7500000
Other,Engineering Research Support Organization,5784169
BP Group,Energy Biosciences Institute,5000000
Other,Chemistry Department,4222588
Intel Corporation,Electrical Engineering & Computer Sciences,3929000
Other,The California Institute for Quantitative Biosciences,3115208
"Pfizer, Inc.",The California Institute for Quantitative Biosciences,2904754
Chevron Corporation,Chemical and Biomolecular Engineering,2825291


In [26]:
# Other mapping 

# Building json file 
# data = {}
# data['key'] = 'value'
# json_data = json.dumps(data)
# 'key' in (data.keys())
industry_other_grouped = industry_other.group('Department', lambda x: x)
industry_other_grouped = industry_other_grouped.with_column('tuples', 
                            industry_other_grouped.apply(lambda s, t: list(zip(s, t)), [1, 2]))
industry_other_grouped.show(4)

Department,Sponsor,Total,tuples
Berkeley Seismological Laboratory,"['Array Information Technology, Inc.']",[213855],"[('Array Information Technology, Inc.', 213855)]"
Bioengineering,['Novartis Institutes for BioMedical Research' 'Zymergen ...,[377099 125600 22500 15650],"[('Novartis Institutes for BioMedical Research', 377099) ..."
Boalt Law School,['Wellspring Advisors LLC'],[70000],"[('Wellspring Advisors LLC', 70000)]"
Botanical Garden,['Vollmar Natural Lands Consulting' 'Creekside Center f ...,[41000 11200],"[('Vollmar Natural Lands Consulting', 41000), ('Creeksid ..."


In [ ]:
# Other mapping 

t1 = industry_other_grouped.column(1)[3]
t2 = industry_other_grouped.column(2)[3]
# for elem in zip(t1, t2):
#     print(elem)
for pair in list(zip(t1, t2)):
    print(pair)
# {"Botanical Garden": [{'sponsor':'Vollmar Natural Lands Consulting', 'value': 41000}, ....]}

In [ ]:
# Other mapping 

other_mapping = {}
targets = industry_other_grouped.column('Department')
tuples = industry_other_grouped.column('tuples')
for i in range(industry_other_grouped.num_rows):
    submapping = []
    for pair in tuples[i]:
        sub = {}
        sub['sponsor'] = pair[0]
        sub['value'] = pair[1]
        submapping += [sub]
    other_mapping[targets[i]] = submapping
other_mapping

In [15]:
def nodes_links(table):
    # NODES
    industry_json = {}
    nodes = []
    links = []
    nodes_details = {}
    sponsors = table.column('Sponsor')
    depts = table.column('Department')
    total = table.column('Total')
    index = 0
    for i in range(table.num_rows):
        node_json = {}
        if sponsors[i] not in nodes_details.keys():
            nodes_details[sponsors[i]] = index
            node_json['name'] = sponsors[i]
            node_json['cat'] = ""
            index += 1
            nodes = nodes + [node_json]
        node_json1 = {}
        if depts[i] not in nodes_details.keys():
            nodes_details[depts[i]] = index
            node_json1['name'] = depts[i]
            node_json1['cat'] = ""
            index += 1
            nodes = nodes + [node_json1]
    # LINKS
    for i in range(table.num_rows):
        link_json = {}
        link_json['source'] = nodes_details[sponsors[i]]
        link_json['target'] = nodes_details[depts[i]]
        link_json['value'] = total[i]
        links += [link_json]
    industry_data = {}
    industry_data['nodes'] = nodes
    industry_data['links'] = links
    return industry_data

In [34]:
# LINKS
# for i in range(industry_all.num_rows):
#     link_json = {}
#     link_json['source'] = nodes_details[sponsors[i]]
#     link_json['target'] = nodes_details[depts[i]]
#     link_json['value'] = total[i]
#     links += [link_json]

In [36]:
# industry_data = {}
# industry_data['nodes'] = nodes
# industry_data['links'] = links
# industry_json_data = json.dumps(industry_data)

In [37]:
industry_data

{}

In [16]:
nodes_links(industry_all)

{'links': [{'source': 0, 'target': 1, 'value': 9069312},
  {'source': 2, 'target': 3, 'value': 7500000},
  {'source': 4, 'target': 5, 'value': 5000000},
  {'source': 6, 'target': 7, 'value': 3929000},
  {'source': 8, 'target': 9, 'value': 2904754},
  {'source': 10, 'target': 11, 'value': 2825291},
  {'source': 6, 'target': 12, 'value': 2409614},
  {'source': 13, 'target': 14, 'value': 2340000},
  {'source': 15, 'target': 16, 'value': 1879998},
  {'source': 17, 'target': 11, 'value': 1810954},
  {'source': 18, 'target': 9, 'value': 1778482},
  {'source': 19, 'target': 20, 'value': 1725779},
  {'source': 21, 'target': 1, 'value': 1724999},
  {'source': 22, 'target': 23, 'value': 1664983},
  {'source': 24, 'target': 12, 'value': 1640860},
  {'source': 25, 'target': 9, 'value': 1502000},
  {'source': 26, 'target': 27, 'value': 1500000},
  {'source': 28, 'target': 27, 'value': 1500000},
  {'source': 29, 'target': 27, 'value': 1500000},
  {'source': 30, 'target': 27, 'value': 1500000},
  {'s

In [23]:
# Checking data -- something weird happened with industry_all but data in industry_1002.json is accurate 
print(sum(industry_all.where('Department', 'The California Institute for Quantitative Biosciences').column("Total")),
sum(industry.where("Department", "The California Institute for Quantitative Biosciences (QB3)").column("Total")))

9300444 9038444


In [ ]:
nodes_links(industry_all.where('Sponsor', 'Pfizer, Inc.'))

### Stacked Area Chart

In [ ]:
sac = combined_phoebe_conc_pd_ds.group('Sponsor Class', sum).select([0, 6, 7, 8])
sac = sac.with_column('Sponsor Class', sac.apply(lambda x: "Other" if x != "Industry" else "Industry", "Sponsor Class"))
sac = sac.group('Sponsor Class', sum).relabel('2014 sum sum', '2014').relabel('2015 sum sum', '2015').relabel('2016 sum sum', '2016')
sac

In [ ]:
sacdf = sac.to_df()
sacdf.transpose().to_csv('sac_rfund_year.csv')

In [ ]:
sac= Table().read_table('sac_rfund_year.csv')
sac

In [ ]:
sac= Table().read_table('sac_rfund_year.csv')
sac = sac.with_column('Total', sac.column('Industry') + sac.column('Other'))
perc = lambda x, total: (x/total) * 100
sac = sac.with_columns(['Industry', sac.apply(perc, [1, 3]),
                        'Other', sac.apply(perc, [2, 3])])
sac

In [ ]:
# From https://stackoverflow.com/questions/16875546/create-a-100-stacked-area-chart-with-matplotlib
fnx = lambda : np.random.randint(5, 50, 10)
y = np.row_stack((fnx(), fnx(), fnx()))
x = np.arange(10)

# Make new array consisting of fractions of column-totals,
# using .astype(float) to avoid integer division
percent = y /  y.sum(axis=0).astype(float) * 100 

# fig = plt.figure()
# ax = fig.add_subplot(111)

# ax.stackplot(sac.column('Year'), percent)
# ax.set_title('100 % stacked area chart')
# ax.set_ylabel('Percent (%)')
# ax.margins(0, 0) # Set margins to avoid "whitespace"

# plt.show()
percent

In [ ]:
y = np.row_stack((sac.column('Industry'), sac.column("Other")))
x = sac.column('Year')

# Make new array consisting of fractions of column-totals,
# using .astype(float) to avoid integer division

fig = plt.figure()
ax = fig.add_subplot(111)

ax.stackplot(x, y)
ax.set_title('100 % stacked area chart')
ax.set_ylabel('Percent (%)')
ax.margins(0, 0) # Set margins to avoid "whitespace"

plt.show()

In [ ]:
N = 3
menMeans = tuple(sac.column("Other"))
womenMeans = tuple(sac.column("Industry"))
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, menMeans, width)
p2 = plt.bar(ind, womenMeans, width,
             bottom=menMeans)

plt.ylabel('Awards')
plt.title('Industry Awards vs. All Other Awards')
plt.xticks(ind, ('2014', '2015', '2016'))
plt.yticks(np.arange(0, 80, 9))
# plt.legend((p1[0], p2[0]), ('Other', 'Industry'))

# plt.show()

### Non-stem/stem

In [ ]:
# Divisions
stem = ['CNR College of Natural Resources', 'Chemistry Department', 'College of Chemistry',
       'College of Engineering', 'College of Environmental Design', 'Division of Physical Sciences',
       'L&S Biological Sciences', 'School of Information', 'School of Optometry', 
       ]
other = ['Strategic Academic and Facilities Planning', 'UC Library', 
        'Vice Provost Academic Affairs & Faculty Welfare', 'Vice Chancellor - Student Affairs Division',
        'Vice Chancellor - Real Estate', 'Vice Chancellor - Administration', 
        'VP Academic Planning & Fac', 'Office of Equity & Inclusion',
         'EVCP Executive Vice Chancellor & Provost']

In [9]:
industry.show()

Sponsor,Control Unit,Division,Department,Total
BASF Corporation,Colleges,Chemistry Department,Chemistry Department,9069312
Aduro Biotech,L&S College of Letters & Sciences,L&S Biological Sciences,Cancer Research Laboratory,7500000
"BP Group (BP America, BP Exploration, British Petroleum)",Vice Chancellor - Research,VC Res Other Research Units,EBI Energy Biosciences Institute,5000000
Intel Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,3929000
"Pfizer, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),2904754
Chevron Corporation,Colleges,College of Chemistry,Chemical and Biomolecular Engineering,2825291
Intel Corporation,Colleges,College of Engineering,ERSO Engineering Research Support Organization,2409614
AstraZeneca Inc.,L&S College of Letters & Sciences,L&S Biological Sciences,Innovative Genomics Initiative,2340000
Wellspring Advisors LLC,Vice Chancellor - Research,VC Res Other Research Units,Center of Evaluation for Global Action,1879998
GlaxoSmithKline (Incl Smith Kline Beecham),Colleges,College of Chemistry,Chemical and Biomolecular Engineering,1810954


In [10]:
# if division or department includes sciences, chemistry, engineering, transportation, 
def non_stem(div, dept):
    keywords = ['social sciences', 'international studies', 'education', 'labor and employment',
               'law', 'haas', 'regional development', 'humanities']
    div_ = np.any([x in div.lower() for x in keywords])
    dept_ = np.any([x in dept.lower() for x in keywords])
    return div_ or dept_

In [14]:
stem_industry = industry.with_column('Non_Stem',
                                    industry.apply(non_stem, ['Division', 'Department']))
stem_industry.where('Non_Stem', True).num_rows

18

In [19]:
non = sum(stem_industry.where('Non_Stem', True).column('Total'))
stem = sum(stem_industry.where('Non_Stem', False).column('Total'))
print(non, "to non-stem from industry")
print(stem, "to stem from industry")
print(stem/non ,": 1 stem to non-stem ratio")

1650620 to non-stem from industry
120856687 to stem from industry
73.2189643891 : 1 stem to non-stem ratio


### Other analysis on private funding

In [27]:
industry.where("Total", are.below(0)).group('Department', sum).sort("Total sum", descending=False)
# Null is School of Optometry

Department,Sponsor sum,Control Unit sum,Division sum,Total sum
ERSO Engineering Research Support Organization,,,,-3786638
Null,,,,-511456
CEDR Center for Environmental Design Research,,,,-324000
The California Institute for Quantitative Biosciences (QB3),,,,-262000
Nutritional Science & Toxicology,,,,-156647
